In [8]:
path_root = r"C:/Users/jgpg000.edu/Desktop/Agente inteligente recuperación sensórica IoT/"

# ruta clases personalizadas
import sys
path_classes = f"{path_root}classes"
sys.path.append(path_classes)

## LIBRERIAS
import pandas as pd
from class_data import *
from fun_models import *

## Agente inteligente

#### Cargar base de datos

In [9]:
path_excel = f"{path_root}datasets/database_items/item_list.xlsx"
path_saved_json = f"{path_root}datasets/database_items/database_json/"

documentos = load_documents(path_excel, path_saved_json)
documentos_langchain = load_documents_langchain(documentos)

from langchain_text_splitters import RecursiveCharacterTextSplitter
# Aplicamos el splitter de Langchain
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

# Dividimos cada texto en chunks
chunks_texto = text_splitter.split_documents(documentos_langchain)
print(f"Hay {len(chunks_texto)} chunks de texto.")

100%|██████████| 88/88 [00:00<00:00, 196.38it/s]


Se han cargado 86 documentos.


100%|██████████| 86/86 [00:00<00:00, 3285.37it/s]


Se han generado 954 documentos de Langchain.
De ellos, 86 son el texto y 868 son resúmenes de tablas.
Hay 7218 chunks de texto.


#### Modelo RAG

In [10]:
path_saved_vector = f"{path_root}datasets/database_items/database_vectorial/"
name_database = "large_esp"
modelo_llm = "gpt-4o"

template_sistema = """ Eres un asistente de una empresa encargada de smart cities.
Como preguntas, te van a adjuntar listas de requisitos de equipo para proyecto.
Tu labor es responder con el nombre de los equipos que encuentres que cumplan los requisitos, así como suministrar la información de estos dispositivos en relación a los requisitos.
Usa la información añadida en el contexto para responder a la pregunta pero no hagas referencias al mismo.
Se añade como metadato tanto el proveedor como el nombre a los que pertenece el contexto devuelto.
Responde únicamente con la especificación que se pide. Si no aparece di que no lo sabes.
Pregunta: {question}
Metadato: {metadata}
Contexto: {contexts}
La respuesta debe tener el siguiente formato:
- Nombre del equipo y proveedor.
- Especificaciones que se han preguntado y aparecen en el contexto.
Respuesta:
"""

vectorstore = create_database(f"{path_saved_vector}{name_database}",
                               chunks_texto, nombre_embedding = "text-embedding-3-large")
retriever = create_retriever(vectorstore)
rag = create_RAG(retriever=retriever,
                plantilla_RAG=template_sistema,
                model=modelo_llm)


Espacio creado correctamente. Contiene 7218 vectores.
Retriever creado correctamente.
RAG creado correctamente.
    Llámalo con <rag_name>.invoke(<query>)


In [2]:
pregunta_ejemplo = '''Se requiere de un sonómetro clase II LoRa que disponga de  las siguientes especificaciones como mínimo:
- Rango: 40 dB a 115 dB. 
- Frecuencia de ponderación A.
- Capacidad para proveer diferentes ponderaciones para el correcto estudio del ruido: LAeq, 
LA1, LA10, LA50, LA90, LA99, LAmax, LAmin, LA Slow o LA Fast entre otros.
- Detección de picos.
- Consumo: 40 mA.
- Protector contra viento.
- Voltaje de operación: 3.6 a 6V.
- Normativa IEC 61672-1.
- Certificación: ROHS / CE.
'''

rag.invoke(pregunta_ejemplo)

NameError: name 'rag' is not defined

#### Interfaz gráfica

In [ ]:
import gradio as gr

def predict(message, history):
    response = rag.invoke(message)
    return response['answer']

gr.ChatInterface(predict).launch()

## Respuestas a Dataset

In [ ]:
path_dataset = f"{path_root}datasets/test/QA_Dataset.xlsx"
qa = QADataset(path_dataset, sheet_name="Hoja1")
qa.show()[:3]

In [ ]:
res = qa.generar_respuesta_individual(rag, 0)
res

In [ ]:
qa.generar_respuestas(rag)
qa.show()[:4]

In [ ]:
## GUARDAMOS EL DATASET CON LAS RESPUESTAS GENERADAS POR EL MODELO
## formato nombre: QA-modelo-Q#: QA-(modeloRAG)-(num. preg. dataset)
path_save_dataset = f"{path_root}testing/responses_test/csv/"
model_name = "QA-baseline_esp_large-Q16"

qa.guardar(path_save_dataset, model_name) # guarda en csv

In [ ]:
path_save_dataset = f"{path_root}testing/responses_test/csv/"
model_name = "QA-baseline_esp_large-Q16"
## Comprobar que se ha guardado correctamente (cargando el modelo)
qa = QADataset(f"{path_save_dataset}{model_name}.csv")
qa.show_metrics()[:3]

#### Evaluamos la calidad de las respuestas generadas

In [ ]:
##### LISTA METRICAS #####
# ['answer_relevancy', 'context_relevancy', 'context_recall', 'answer_similarity',
# 'answer_correctness', 'precision_bertscore', 'recall_bertscore', 'f1_bertscore',
# 'mismo_producto', 'especificaciones_correctas', 'especificaciones_correctas_norm']

In [ ]:
from ragas.metrics import context_relevancy, answer_similarity, answer_correctness
metricas = [context_relevancy, answer_similarity, answer_correctness]

# generamos todas las métricas con el siguiente bucle
result_ragas = qa.ragas(metricas= metricas)
qa.bertscore()
qa.es_mismo_producto(modelo_llm="gpt-4o")
qa.son_especificaciones_correctas(modelo_llm="gpt-4o")

# Guardamos el dataset con métricas
path_save_dataset = f"{path_root}testing/response_test/csv/"
model_name = "QA-baseline_esp_large-Q16"
qa.guardar(path_save_dataset, model_name)

In [ ]:
# Incluimos la métrica del número de especificaciones correctas normalizadas
path_dataset = f"{path_root}datasets/test/QA_Dataset.xlsx"
dataset = pd.read_excel(path_dataset, sheet_name="QA2", dtype=str, header=0)
num_especificaciones = dataset['nº ENCONTRADAS EN DATASHEET'].dropna().astype(int).reset_index(drop=True)

qa.son_especificaciones_correctas_norm(num_especificaciones)
qa.guardar(path_save_dataset, model_name)
qa.especificaciones_correctas_norm[:3]

La QA correcta 14 tiene tantas especificaciones_correctas sin acertar el producto porque la mayoría de la ground_truth es "no lo sé".

In [ ]:
metricas = QADataset.columnas_metricas.copy()
metricas.remove("especificaciones_correctas")
metricas.remove("answer_relevancy")
metricas.remove("context_recall")

path_figures = f"{path_root}testing/figures_test/"
model_name = "QA-baseline_esp_large-Q16"
qa.grafico(model_name, path_figures, metricas = metricas)